# Insert Seed
read the seed account usernames from the `config.json` file, get their ids and account details and then insert them into the db

In [1]:
# required imports to access api_db, misc, misc.CONFIG, ...
import sys
sys.path = ['.', '..', '../..'] + sys.path
from collection import *

using 128 seed accounts and 0 hashtags
Database stats: 
{'avgObjSize': 0.0,
 'collections': 0,
 'dataSize': 0.0,
 'db': 'electionswatch',
 'fsTotalSize': 510770802688.0,
 'fsUsedSize': 450388680704.0,
 'indexSize': 0.0,
 'indexes': 0,
 'numExtents': 0,
 'objects': 0,
 'ok': 1.0,
 'scaleFactor': 1.0,
 'storageSize': 0.0,
 'views': 0}
DB size (B): 0.0
DB size (MB): 0.00
DB size (GB): 0.00
Using API keys for app twitta-mos

Done initializing at 05:07PM on September 19, 2020.
----------------------------------------


<hr>
<h1 align="center">driver code</h1>

In [2]:
seed_in_db = api_db.col_users.find(
    {"depth":0, "screen_name": {"$exists": True}},
    {"screen_name": True, "_id": False}, no_cursor_timeout=True)
already_collected = [u["screen_name"] for u in seed_in_db] # typically this will not be too large
print("Found %d accounts already collected" % len(already_collected))

Found 0 accounts already collected


In [3]:
usernames = misc.CONFIG["seed"]["usernames"]
new_usernames = list(set(usernames) - set(already_collected))
print("Only %d accounts left to fetch and insert" % len(new_usernames))
seed = get_users_from_usernames(usernames)
# if an error occurs it is logged to stdout, if number don't match then it is just a duplicate
print("got %d/%d accounts (%d not found, suspended or duplicate)" % (len(seed), len(usernames), len(usernames) - len(seed)))

Only 127 accounts left to fetch and insert
got 127/128 accounts (1 not found, suspended or duplicate)


In [4]:
# convert to db format
seed = list(map(user_to_db_format, seed))

In [5]:
# set their depth=0
for s in seed: s["depth"] = 0

In [6]:
upsert_users(seed)

In [7]:
print("DONE")

DONE
